In [1]:
pip install pinecone-client

Note: you may need to restart the kernel to use updated packages.


In [2]:
  !pip install -qU \
  transformers==4.31.0 \
  sentence-transformers==2.2.2 \
  datasets==2.14.0 \
  accelerate==0.21.0 \
  einops==0.6.1 \
  langchain==0.0.240 \
  xformers==0.0.20 \
  bitsandbytes==0.41.0\
  InstructorEmbedding

In [3]:
pip install ctransformers

In [4]:
pip install googletrans==4.0.0-rc1

Note: you may need to restart the kernel to use updated packages.


In [18]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
import tempfile
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.chat_models import ChatOpenAI
from transformers import AutoTokenizer
import os
import pandas as pd

In [8]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY','2693584c-f620-496b-b7db-40232853ebdd')
PINECONE_API_ENV=os.environ.get('PINECONE_API_ENV', 'gcp-starter')

In [9]:
index_name = "medbot"

In [46]:
path='chatbot.csv'

In [47]:
loader = CSVLoader(file_path=path)
data = loader.load()

In [50]:
document_texts = [doc.page_content for doc in data]

In [51]:
input_text = "\n".join(document_texts)
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
tokenized_text = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512, padding="max_length")
input_ids = tokenized_text["input_ids"]
input_text = tokenizer.decode(input_ids[0])

In [53]:
# Assuming 'documents' is a list of strings from the text file

# Initialize a list to store the split documents
split_docs = []

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

# Split each line/document into smaller chunks
for document in document_texts:
    split_chunks = text_splitter.split_text(input_text)
    split_docs.extend(split_chunks)

# Now, 'split_docs' contains the smaller text chunks

In [54]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

C:\Users\navya\anaconda3\Lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [55]:
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

In [57]:
docsearch = Pinecone.from_texts(split_docs, embeddings, index_name=index_name)

KeyboardInterrupt: 

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that I don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [ ]:
chain_type_kwargs={"prompt": PROMPT}

llm=CTransformers(model="llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [ ]:
qa=RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever(search_kwargs={'k': 1}),return_source_documents=True, chain_type_kwargs=chain_type_kwargs)

In [ ]:
user_input = input(f"Input Prompt:")
result=qa({"query": user_input})
print("Response : ", result["result"])